Much of the code was appropriated for my purposes from this video:
https://youtu.be/8QuJKESRuCw DJ Oamen

In [1]:
from tkinter import*
from tkinter import ttk
import random

import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [2]:
# database setup
engine = create_engine("sqlite:///resources/vgsales_db.sqlite")

# reflect database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# assign the 'vgsales' class to a variable called 'vgsales'
vgsales = Base.classes.vgsales

In [3]:
class dbConnect:
    def __init__(self, root):
        self.root = root
        blankSpace = " "
        self.root.title(155 * blankSpace + "SQLite.Video Game Sales")
        self.root.resizable(width=False, height=False)
        self.root.geometry("1110x600+0+0")
        
        Search = StringVar()
        
        # first frame created: initial internal area created inside of root parent
        MainFrame = Frame(self.root, bd=10, width=1105, height=600, relief=RIDGE, bg='aliceblue')
        # grid is fine instead of place, not resizing the window
        MainFrame.grid()
        
        # title bar and search bar frames created (second and third frames)
        TitleFrames = Frame(MainFrame, bd=7, width=1080, height=100, relief=RIDGE)
        TitleFrames.grid(row=0, column=0)
        TitleFrame = Frame(TitleFrames, bd=7, width=1080, height=100, relief=RIDGE, bg='aliceblue')
        TitleFrame.grid(row=0, column=0)
        
        SearchFrames = Frame(MainFrame, bd=5, width=1094, height=94, relief=RIDGE)
        SearchFrames.grid(row=1, column=0)
        
        # lower frame space for the table, fourth frame created (third inside main)
        MidFrame = Frame(MainFrame, bd=5, width=1088, height=420, relief=RIDGE)
        MidFrame.grid(row=3, column=0)
        
        # create a frame for the table inside the lower MidFrame
        TreeFrame = Frame(MidFrame, bd=5, width=1088, height=400, relief=RIDGE)
        TreeFrame.grid(row=0, column=0,)
        
        # widget for the title, inside the title bar frame
        self.iblTitle = Label(TitleFrames, font=('arial', 40, 'bold'),
                              text='Video Game Sales', bg='aliceblue', fg='cornflowerblue')
        self.iblTitle.grid(row=0, column=0)
        
        # create search entry form, search button, and reset button; inside the search frame
        self.txtSearch = Entry(SearchFrames, font=('arial', 20), width=46, justify='left')#, textvariable=Search
        self.txtSearch.grid(row=0, column=0)
        self.btnSearch = Button(SearchFrames, bd=4, font=('arial', 13, 'bold'), text="SEARCH",
                                width=18, height=1, bg='cornflowerblue', fg='white')
        self.btnSearch.grid(row=0, column=2)
        self.btnReset = Button(SearchFrames, bd=4, font=('arial', 13, 'bold'), text="RESET",
                                width=18, height=1, bg='cornflowerblue', fg='white')
        self.btnReset.grid(row=0, column=3)
        
         # first function created to query SQLite database on launch or reset
        def showRecord():
            # create a session
            session = Session(engine)
            # query all data
            results = session.query(vgsales).all()
            # if results exist, refresh the table...
            if len(results) != 0:
                self.game_records.delete(*self.game_records.get_children())
                # ...and insert the data with a for loop
                for row in results:
                    self.game_records.insert('', 'end', values=(row.rank, row.name, row.platform,
                                                               row.year, row.genre, row.publisher, row.na_sales,
                                                               row.eu_sales, row.jp_sales, row.other_sales, row.global_sales))
            # close session
            session.close()
           
        # create second function to filter for results based on search
        def getRecord(query):
            session = Session(engine)
            results = session.query(vgsales).filter((vgsales.genre == f'{query}') | (vgsales.platform == f'{query}')\
                                                    | (vgsales.year == f'{query}') | (vgsales.publisher == f'{query}')).all()
            if len(results) != 0:
                self.game_records.delete(*self.game_records.get_children())
                for row in results:
                    self.game_records.insert('', 'end', values=(row.rank, row.name, row.platform,
                                                               row.year, row.genre, row.publisher, row.na_sales,
                                                               row.eu_sales, row.jp_sales, row.other_sales, row.global_sales))
            session.close()
        
        # widget for scrollbar and table inside the TreeFrame
        scroll_y = Scrollbar(TreeFrame, orient=VERTICAL)
        
        self.game_records = ttk.Treeview(TreeFrame, height=19, columns=("rank", "name", "platform", "year", "genre",
                                                                        "publisher", "na_sales", "eu_sales", "jp_sales",
                                                                        "other_sales", "global_sales"), yscrollcommand=scroll_y.set)
        scroll_y.pack(side=RIGHT, fill=Y)
        
        self.game_records.heading("rank", text="Rank")
        self.game_records.heading("name", text="Name")
        self.game_records.heading("platform", text="Platform")
        self.game_records.heading("year", text="Year")
        self.game_records.heading("genre", text="Genre")
        self.game_records.heading("publisher", text="Publisher")
        self.game_records.heading("na_sales", text="NA Sales")
        self.game_records.heading("eu_sales", text="EU Sales")
        self.game_records.heading("jp_sales", text="JP Sales")
        self.game_records.heading("other_sales", text="Other Sales")
        self.game_records.heading("global_sales", text="Global Sales")
        
        self.game_records['show']='headings'
        
        # describing column width
        self.game_records.column("rank", width=48)
        self.game_records.column("name", width=217)
        self.game_records.column("platform", width=68)
        self.game_records.column("year", width=48)
        self.game_records.column("genre", width=76)
        self.game_records.column("publisher", width=144)
        self.game_records.column("na_sales", width=91)
        self.game_records.column("eu_sales", width=91)
        self.game_records.column("jp_sales", width=91)
        self.game_records.column("other_sales", width=91)
        self.game_records.column("global_sales", width=91)
        
        self.game_records.pack(fill=BOTH, expand=1)
        self.game_records.bind("<ButtonRelease-1>")
        
        # configure the scrollbar
        scroll_y.config(command=self.game_records.yview)
        
        # configure search and reset buttons
        self.btnSearch.config(command=lambda: getRecord(self.txtSearch.get()))
        self.btnReset.config(command=lambda: showRecord())
        
        showRecord()

In [4]:
if __name__=='__main__':
    root = Tk()
    application = dbConnect(root)
    root.mainloop()